<a href="https://colab.research.google.com/github/ShreyaPMohanty6/Lunar-Landing/blob/main/Deep_Q_Learning_for_Lunar_Landing_Shreya_Mohanty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.2 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446659 sha256=a4293f85d1330b335496e7f73048887018df9ee7b575e23406594672a0968269
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed

### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class neural_network(nn.Module):
    def __init__(self, state_size, actions_size, seed = 42):
        super(neural_network, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, 64)  #Connection between the input layer and first fully connected layer
        self.fc2 = nn.Linear(64, 64)          #Connection between the fc1 and fc2
        self.fc3 = nn.Linear(64, actions_size) #Connection between the fc2 and action layer

    #building Signal that will propagate through the layers
    def forward(self, state):
        x = F.relu(self.fc1(state))  #By rectifier activation function
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Part 2 - Training the AI

### Setting up the environment

In [ ]:
import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
print("State Shape", state_shape)
print("State Size", state_size)
print("Action Size", action_size)

State Shape (8,)
State Size 8
Action Size 4


### Initializing the hyperparameters

In [ ]:
learning_rate = 5e-4
discount_factor = 0.99
buffer_size = 100000
minibatch_size = 100
interpolation_parameter = 1e-3

### Implementing Experience Replay

In [ ]:
class Replaymemory(object):
  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
       del self.memory[0]

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

### Implementing the DQN class

In [ ]:
class Agent():
  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    #Implementing Q learning, comparing Q target and Q local
    self.local_Qnetwork = neural_network(state_size, action_size).to(self.device)  #Local Q network will be the one selecting the actions to be implemneted and target Q values are use to train the local Q values
    self.target_Qnetwork = neural_network(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_Qnetwork.parameters(), lr = learning_rate)
    self.memory = Replaymemory(buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4 #to repeat every 4 times

# Now learing from experience in every 4 steps
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size: #learn from these experiences
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor) #The learning algorithm which is used later on

#Act on the basis of the given action in the environment and a given epislon value using epsilon greedy method
  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device) #to add the batch number of the state within the batch of observations
    self.local_Qnetwork.eval() #Setting local Q value to evalution mode
    with torch.no_grad():
      action_values = self.local_Qnetwork(state)  #making predictions from our Q local netwrok to get the action values
    self.local_Qnetwork.train() #setting back to training mode

#Epilson Greedy Selection Policy Implementation
    if random.random() > epsilon:
       return np.argmax(action_values.cpu().data.numpy())
    else:
       return random.choice(np.arange(self.action_size))

#learning algorithm
  def learn(self, experiences, gamma):
    states, next_states, actions, rewards, dones = experiences
    #to get the maximum predicted Q values for next states
    next_q_targets = self.target_Qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)  #Formula for calculating current Q target
    q_expected = self.local_Qnetwork(states).gather(1, actions)    #forward propagating the states from local Q/ Getting Q values from local Q values
    loss = F.mse_loss(q_expected, q_targets)   #Calculating the loss from both the values
    self.optimizer.zero_grad()
    #backpropagating the loss computed to compute the gradient of the loss
    q_values = self.local_Qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_values, q_targets)
    loss.backward()
    #single optimization step to update the model parameters
    self.optimizer.step()
    #updating the targetting Q values with the local ones
    self.soft_update(self.local_Qnetwork, self.target_Qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, tau):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

### Initializing the DQN agent

In [ ]:
agent = Agent(state_size, action_size)

### Training the DQN agent

In [ ]:
number_episodes = 2000  #number of episodes over which the agent will be trained
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

#Building the intelligence to land on the moon
for episode in range(1, number_episodes+1):
  state, _ = env.reset()  #reseting the environment for every new episode
  score = 0 #Score is the culmulative rewards collected
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon) #Action to be selected
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)  #this is the actual step which will train the agnet
    state = next_state
    score += reward
    if done:
      break

  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon) #Decaying the epsilon value

  #printing the values
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "") #\r is the dynamic affect which will overreide the info of the prvious episode
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_Qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -97.66
Episode 200	Average Score: -12.22
Episode 300	Average Score: 88.32
Episode 400	Average Score: 194.36
Episode 409	Average Score: 200.61
Environment solved in 309 episodes!	Average Score: 200.61


## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()